In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

nazwa_pliku = 'sensor_readings_24.data'

try:

    df = pd.read_csv(nazwa_pliku, sep=',', header=None)
    
    nowe_nazwy = [f"US{i}" for i in range(1, 25)]
    nowe_nazwy.append('Class')
    df.columns = nowe_nazwy
    
    print("\n--- Informacje o wymiarach ---")
    wiersze, kolumny = df.shape
    print(f"Liczba wierszy: {wiersze}")
    print(f"Liczba kolumn: {kolumny}")
    
    print("\n--- Analiza kolumny class ---")
    liczba_unikalnych = df.iloc[:, -1].nunique()
    print(f"Liczba unikalnych wartości w ostatniej kolumnie: {liczba_unikalnych}")
    
    print(f"Te wartości to: {df.iloc[:, -1].unique()}")
    
    print("\n" + "="*40)
    print("ROZKŁAD KLAS W CAŁYM ZBIORZE:")
    print("="*40)
    
    counts = df['Class'].value_counts()
    percents = df['Class'].value_counts(normalize=True) * 100
    
    for klasa in counts.index:
        liczba = counts[klasa]
        procent = percents[klasa]
        print(f"-- {klasa}: {liczba} samples ({procent:.2f}%).")

    df['Set'] = ""
    
    train_indices, test_indices = train_test_split(
        df.index, 
        test_size=0.2, # dla proporcji 80/20
        stratify=df['Class'], # dla zachowania proporcjonalnego rozkładu klas w zbiorach train i test
        random_state=42 # inaczej seed
    )
    
    df.loc[train_indices, 'Set'] = 'train'
    df.loc[test_indices, 'Set'] = 'test'

    print("\n--- Podgląd po zmianach (pierwsze 5 wierszy) ---")
    print(df.head())
    
    
    print("\n" + "="*40)
    print("PODSUMOWANIE PODZIAŁU (TRAIN vs TEST):")
    print("="*40)
    
    set_counts = df['Set'].value_counts()
    set_percents = df['Set'].value_counts(normalize=True) * 100
    
    for nazwa_setu in set_counts.index:
        liczba = set_counts[nazwa_setu]
        procent = set_percents[nazwa_setu]
        print(f"Zbiór '{nazwa_setu}': {liczba} wierszy ({procent:.2f}% całości)")

    # Dodatkowe sprawdzenie: Czy proporcje klas zostały zachowane wewnątrz zbiorów?
    print("\n--- Weryfikacja proporcji klas w poszczególnych zbiorach (%) ---")
    proporcje = pd.crosstab(df['Class'], df['Set'], normalize='columns') * 100
    print(proporcje.round(2))
    
    df.to_excel("sensor_readings_24.xlsx", index=False)
    print("\n--> Zapisano gotowy plik: sensor_readings_24.xlsx")
    

except FileNotFoundError:
    print("Nie znaleziono pliku. Sprawdź czy nazwa i ścieżka są poprawne.")
except Exception as e:
    print(f"Wystąpił błąd: {e}")


--- Informacje o wymiarach ---
Liczba wierszy: 5456
Liczba kolumn: 25

--- Analiza kolumny class ---
Liczba unikalnych wartości w ostatniej kolumnie: 4
Te wartości to: ['Slight-Right-Turn' 'Sharp-Right-Turn' 'Move-Forward' 'Slight-Left-Turn']

ROZKŁAD KLAS W CAŁYM ZBIORZE:
-- Move-Forward: 2205 samples (40.41%).
-- Sharp-Right-Turn: 2097 samples (38.43%).
-- Slight-Right-Turn: 826 samples (15.14%).
-- Slight-Left-Turn: 328 samples (6.01%).

--- Podgląd po zmianach (pierwsze 5 wierszy) ---
     US1    US2    US3    US4  US5    US6  US7    US8    US9   US10  ...  \
0  0.438  0.498  3.625  3.645  5.0  2.918  5.0  2.351  2.332  2.643  ...   
1  0.438  0.498  3.625  3.648  5.0  2.918  5.0  2.637  2.332  2.649  ...   
2  0.438  0.498  3.625  3.629  5.0  2.918  5.0  2.637  2.334  2.643  ...   
3  0.437  0.501  3.625  3.626  5.0  2.918  5.0  2.353  2.334  2.642  ...   
4  0.438  0.498  3.626  3.629  5.0  2.918  5.0  2.640  2.334  2.639  ...   

    US17   US18   US19   US20   US21   US22   US

## Nasze wczytanie danych itp

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Wczytanie danych z gotowym podziałem train/test
df = pd.read_excel("sensor_readings_24.xlsx")

features = [f"US{i}" for i in range(1, 25)]

# Podział na zbiory według kolumny 'Set'
X_train = df[df['Set']=='train'][features].values
X_test = df[df['Set']=='test'][features].values

y_train_labels = df[df['Set']=='train']['Class'].values
y_test_labels = df[df['Set']=='test']['Class'].values

# LabelEncoder dla zamiany nazw klas na liczby
le = LabelEncoder()
y_train_int = le.fit_transform(y_train_labels)
y_test_int = le.transform(y_test_labels)

# One-hot encoding dla sieci
num_classes = len(le.classes_)
y_train = np.eye(num_classes)[y_train_int]
y_test = np.eye(num_classes)[y_test_int]

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Funkcja aktywacji: Sigmoid
def sigmoid(x, derivative=False):
    if derivative:
        return x * (1 - x)
    return 1 / (1 + np.exp(-x))

def softmax(x):
    exps = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exps / np.sum(exps, axis=1, keepdims=True)

def relu(x, derivative=False):
    if derivative:
        return (x > 0).astype(float)
    return np.maximum(0, x)

# Inicjalizacja wag sieci dla wielu warstw ukrytych
# def initialize_weights(input_size, hidden_layers_sizes, output_size):
#     weights = []
#     layer_sizes = [input_size] + hidden_layers_sizes + [output_size]
#     for i in range(len(layer_sizes) - 1):
#         weights.append(2 * np.random.random((layer_sizes[i], layer_sizes[i+1])) - 1)
#     return weights

def initialize_weights(input_size, hidden_layers_sizes, output_size):
    weights = []
    layer_sizes = [input_size] + hidden_layers_sizes + [output_size]
    for i in range(len(layer_sizes) - 1):
        weights.append(np.random.randn(layer_sizes[i], layer_sizes[i+1]) * np.sqrt(2 / layer_sizes[i]))
    return weights

# Podział danych na zbiory treningowe, generalizacyjne i walidacyjne
def split_data(data, train_ratio=0.6, validation_ratio=0.2):
    np.random.shuffle(data) # tasowanie danych
    
    train_size = int(len(data) * train_ratio) 
    validation_size = int(len(data) * validation_ratio)

    train_data = data[:train_size] # wybiera obserwacje do liczby "train_size"
    validation_data = data[train_size:train_size + validation_size] # wybiera obserwacje od "train_size" do sumy "train_size" i "validation_size"
    test_data = data[train_size + validation_size:] # wybiera obserwacje od powyzszej sumy do końca

    return train_data, validation_data, test_data

# Funkcja dostosowująca tempa nauki
def adjust_learning_rate(learning_rate, mse, previous_mse, learning_rate_adjust, threshold=0.001):
    if mse < previous_mse:
        learning_rate *= 1.1  # jeśli błąd maleje, to zwiększa współczynnik uczenia o 10%
    else:
        learning_rate *= 0.5  # jeśli błąd nie maleje, to zmniejszamy współczynnik uczenia o 50%

    if np.abs(mse - previous_mse) < threshold: #jeśli różnica między błędami jest mniejsza niż dany próg to zmniejszamy learning rate, bo zbliżamy się do optymalnej konfiguracji
        learning_rate *= learning_rate_adjust

    return learning_rate

# Trening sieci z wieloma warstwami ukrytymi
def train(X, y, learning_rate, learning_rate_adjust, epochs,
          hidden_layers_sizes, optimizer, momentum):

    input_size = X.shape[1]
    output_size = y.shape[1]
    weights = initialize_weights(input_size, hidden_layers_sizes, output_size)
    velocities = [np.zeros_like(w) for w in weights]


    for epoch in range(epochs):
        # Forward pass
        activations = [X]
        zs = []
        for i, w in enumerate(weights):
            z = np.dot(activations[-1], w)
            if i == len(weights) - 1:
                activations.append(softmax(z))
            else:
                activations.append(relu(z))
        predicted_output = activations[-1]

        # Backpropagation
        error = predicted_output - y  # cross-entropy gradient
        deltas = [error] 
        for i in range(len(weights) - 1, 0, -1):
            delta = deltas[-1].dot(weights[i].T) * relu(activations[i], derivative=True)
            deltas.append(delta)

        deltas.reverse() 

        # Aktualizacja wag
        # for i in range(len(weights)):
        #     weights[i] += activations[i].T.dot(deltas[i]) * learning_rate

        # # Dostosowanie learning rate
        # learning_rate = adjust_learning_rate(
        #     learning_rate,
        #     np.mean(error ** 2),
        #     np.mean((y - predicted_output) ** 2),
        #     learning_rate_adjust
        # )

        # for i in range(len(weights)):
        #     weights[i] -= learning_rate * activations[i].T.dot(deltas[i]) / X.shape[0]
        for i in range(len(weights)):
            grad = activations[i].T.dot(deltas[i]) / X.shape[0]

            if optimizer == 'gd':
                weights[i] -= learning_rate * grad

            elif optimizer == 'momentum':
                velocities[i] = momentum * velocities[i] - learning_rate * grad
                weights[i] += velocities[i]

    return weights


def predict(X, weights):
    output = X
    for i, w in enumerate(weights):
        output = np.dot(output, w)
        if i < len(weights) - 1:
            output = relu(output)
        else:
            output = softmax(output)
    return output

# Funkcja obliczająca błąd predykcji
def calculate_error(predictions, labels):
    return np.mean(np.abs(predictions - labels))

def correct(y, predictions):
    # y i predictions są one-hot encoded
    y_class = np.argmax(y, axis=1)
    pred_class = np.argmax(predictions, axis=1)
    correct_predictions = np.sum(y_class == pred_class)
    return correct_predictions / len(y)

# Parametry sieci
learning_rates = [0.01]
learning_rate_adjusts = [0.0005]
epochses = [1000]
repeat = 3
optimizers = ['gd', 'momentum']
momentums = [0.0, 0.9]
# gd → zwykły gradient prosty
# momentum → gradient z momentem

# Warstwy
hidden_layers_sizes_list = [
    [24],         
    [24, 10]       
]

# Funkcja obliczająca wyniki dla accuracy, precision, recall, F1
def calculate_metrics(y_test_class, y_pred_class):
    accuracy = accuracy_score(y_test_class, y_pred_class)
    precision = precision_score(y_test_class, y_pred_class, average='macro')
    recall = recall_score(y_test_class, y_pred_class, average='macro')
    f1 = f1_score(y_test_class, y_pred_class, average='macro')
    
    return accuracy, precision, recall, f1

# Przechowywanie wyników dla różnych konfiguracji warstw
results = []

# Testowanie
for hidden_layers_sizes in hidden_layers_sizes_list:  
    for r in range(1, repeat + 1):
        for lr in learning_rates:
            for lr_adj in learning_rate_adjusts:
                for epochs in epochses:
                    for optimizer in optimizers:
                        for momentum in momentums:
                            trained_weights = train(
                                X_train, y_train, lr, lr_adj, epochs, hidden_layers_sizes, optimizer=optimizer, momentum=momentum
                            )
                            predictions_train = predict(X_train, trained_weights)
                            # predictions_validation = predict(X_validation, trained_weights)
                            predictions_test = predict(X_test, trained_weights)

                            # Zaokrąglamy wyniki do wartości 0 lub 1 dla porównań
                            # y_pred = (predictions_test > 0.5).astype(int)
                            y_pred_class = np.argmax(predictions_test, axis=1)
                            y_test_class = np.argmax(y_test, axis=1)

                            # Obliczamy metryki
                            accuracy, precision, recall, f1 = calculate_metrics(y_test_class, y_pred_class)

                            # Dodanie wyników do tabeli
                            results.append({
                                'hidden_layers': str(hidden_layers_sizes),
                                'optimizer': optimizer,
                                'momentum': momentum,
                                'learning_rate': lr,
                                'learning_rate_adjust': lr_adj,
                                'epochs': epochs,
                                'repeat': r,
                                'accuracy': accuracy,
                                'precision': precision,
                                'recall': recall,
                                'f1': f1
                            })

# Tworzenie DataFrame z wynikami
results_df = pd.DataFrame(results)

# Wyświetlanie wyników
print(results_df)
print(results_df.head())

summary = results_df.groupby(
    ['hidden_layers', 'learning_rate', 'optimizer', 'momentum']
).agg(
    avg_accuracy=('accuracy', 'mean'),
    avg_precision=('precision', 'mean'),
    avg_recall=('recall', 'mean'),
    avg_f1=('f1', 'mean'),
    best_accuracy=('accuracy', 'max'),
    best_f1=('f1', 'max')
).reset_index()

print(summary)

   hidden_layers optimizer  momentum  learning_rate  learning_rate_adjust  \
0           [24]        gd       0.0           0.01                0.0005   
1           [24]        gd       0.9           0.01                0.0005   
2           [24]  momentum       0.0           0.01                0.0005   
3           [24]  momentum       0.9           0.01                0.0005   
4           [24]        gd       0.0           0.01                0.0005   
5           [24]        gd       0.9           0.01                0.0005   
6           [24]  momentum       0.0           0.01                0.0005   
7           [24]  momentum       0.9           0.01                0.0005   
8           [24]        gd       0.0           0.01                0.0005   
9           [24]        gd       0.9           0.01                0.0005   
10          [24]  momentum       0.0           0.01                0.0005   
11          [24]  momentum       0.9           0.01                0.0005   

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# =======================
# Parametry symulacji
# =======================
learning_rates = [0.01]
learning_rate_adjusts = [0.5]
epochses = [2000]
repeat = 3
hidden_layers_sizes_list = [[10], [10, 10]]
optimizers = ['gd', 'momentum']
momentums = [0.0, 0.9]

results = []

# =======================
# Wczytanie danych
# =======================
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Wczytanie gotowego pliku z podziałem
df = pd.read_excel("sensor_readings_24.xlsx")

# Cechy
features = [f"US{i}" for i in range(1, 25)]

# Podział na X i y według kolumny 'Set'
X_train = df[df['Set']=='train'][features].values
y_train = df[df['Set']=='train']['Class'].values

X_test = df[df['Set']=='test'][features].values
y_test = df[df['Set']=='test']['Class'].values

# =======================
# LabelEncoder dla etykiet
# =======================
le = LabelEncoder()
y_train_int = le.fit_transform(y_train)
y_test_int = le.transform(y_test)

# =======================
# Tworzymy DataFrame treningowy z X i y razem
# =======================
train_df = pd.DataFrame(X_train, columns=features)
train_df['class'] = y_train_int

# =======================
# Balansowanie zbioru treningowego
# =======================
min_count = train_df['class'].value_counts().min()
balanced_train_df = pd.concat([
    grp.sample(min_count, random_state=42)
    for _, grp in train_df.groupby('class')
])

# Oddzielamy cechy i etykiety po balansowaniu
X_train_balanced = balanced_train_df[features].values
y_train_balanced = balanced_train_df['class'].values

# =======================
# Standaryzacja
# =======================
scaler = StandardScaler()
X_train_balanced = scaler.fit_transform(X_train_balanced)
X_test_scaled = scaler.transform(X_test)  # Test nie jest balansowany

results = []

for lr in learning_rates:
    for lr_adj in learning_rate_adjusts:
        adjusted_lr = lr * lr_adj
        for epochs in epochses:
            for hidden_layers in hidden_layers_sizes_list:
                for r in range(repeat):
                    for optimizer in optimizers:
                        for momentum in momentums:
                            # Dobór parametrów dla MLPClassifier
                            if optimizer == 'gd':
                                clf_momentum = 0.0
                            else:  # 'momentum'
                                clf_momentum = momentum
                            
                            mlp = MLPClassifier(
                                hidden_layer_sizes=hidden_layers,
                                activation='logistic',
                                max_iter=epochs,
                                solver='sgd',
                                learning_rate_init=adjusted_lr,
                                momentum=clf_momentum,
                                random_state=r
                            )
                            mlp.fit(X_train_balanced, y_train_balanced)
                            
                            y_pred = mlp.predict(X_test_scaled)

                            # Obliczanie metryk
                            accuracy = accuracy_score(y_test_int, y_pred)
                            precision = precision_score(y_test_int, y_pred, average='macro', zero_division=0)
                            recall = recall_score(y_test_int, y_pred, average='macro', zero_division=0)
                            f1 = f1_score(y_test_int, y_pred, average='macro', zero_division=0)
                            
                            results.append({
                                "hidden_layers": str(hidden_layers),
                                "optimizer": optimizer,
                                "momentum": momentum,
                                "learning_rate": lr,
                                "learning_rate_adjust": lr_adj,
                                "adjusted_lr": adjusted_lr,
                                "epochs": epochs,
                                "repeat": r+1,
                                "accuracy": accuracy,
                                "precision": precision,
                                "recall": recall,
                                "f1_score": f1
                            })

df_results = pd.DataFrame(results)
print(df_results)

# Opcjonalnie zapis
# df_results.to_csv("mlp_results.csv", index=False)

summary = df_results.groupby(
    ['hidden_layers', 'learning_rate', 'optimizer', 'momentum']
).agg(
    avg_accuracy=('accuracy', 'mean'),
    avg_precision=('precision', 'mean'),
    avg_recall=('recall', 'mean'),
    avg_f1=('f1_score', 'mean'),
    best_accuracy=('accuracy', 'max'),
    best_f1=('f1_score', 'max')
).reset_index()

print("\nPodsumowanie wyników:")
print(summary)


C:\Users\julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\juli